In [1]:
import numpy as np
import torch
from torch import nn
from scipy.integrate import odeint
import pandas as pd
import matplotlib.pyplot as plt
import math

###定义求导/差分运算###
def derivative(t, s):  #使用差分代替导数，可以用记录的位移值代替计算速度
    dsdt = (s[1:] - s[:-1]) / (t[1:] - t[:-1])  #这就意味着每次使用差分求一次导，数组的大小就要-1，s记录n个数据，则v只能记录n-1个数据
    return dsdt 

def load_data(csv_path):  #读取位移和时间数据 
    data = pd.read_csv(csv_path)   
    t = np.array(data["t"])  #读取时间
    s = np.array(data["s"])  #读取位移
    return t, s

###读取数据###
csv_path = "sv_data.csv"   
t, s = load_data(csv_path)  #读取数据
v = derivative(t, s) #计算各个时刻的速度
#print(v)

In [2]:
###定义求解（回归出）微分方程参数的网络###
class StraightLine(nn.Module):  #使用Pytorch搭建微分方程模型
    def __init__(self):
        super().__init__()
        self.alpha = nn.Parameter(torch.randn((1,), requires_grad=True))  #需要记录下来参数，requires_grad = True可以一直不用改
        self.beta = nn.Parameter(torch.randn((1,), requires_grad=True)) 
        
    def forward(self, theta, omega, add_force=False):
        a = - self.alpha * omega - self.beta * theta  #微分方程
        return a

In [3]:
###位移s，速度v，加速度a的数组大小统一，s需要删掉最后两个值，v需要删掉最后一个值，a不需要删###
#为了能够将数据导入到PyTorch中，需要将np.array和list转为torch.tensor，代码如下#
a = torch.from_numpy(derivative(t[:-1], v))  #每次使用差分求一次导，数组的大小就要-1
#print(a)
_v = torch.from_numpy(v[:-1]) #每次使用差分求一次导，数组的大小就要-1,s记录n个数据，则v只能记录n-1个数据,a只能记录n-2个数据
_s = torch.from_numpy(s[:-2]) #每次使用差分求一次导，数组的大小就要-1,s记录n个数据，则v只能记录n-1个数据,a只能记录n-2个数据

In [4]:
###开始进行训练###
num_epoch = 20000  #训练20000轮就差不多了
model = StraightLine()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [5]:
for i in range(num_epoch):
    optimizer.zero_grad()
    loss = torch.mean((a - model(_s, _v)) ** 2)  #最小二乘Loss
    if (i + 1) % 1000 == 0:
        print("{}:{}".format(i + 1, loss.item()))
    loss.backward()
    optimizer.step()

C:\ProgramData\Anaconda3\envs\ai4engineeringcourse\lib\site-packages\torch\autograd\__init__.py:251: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at C:\cb\pytorch_1000000000000\work\c10\cuda\CUDAFunctions.cpp:108.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


1000:7.669797789956363
2000:5.0278741313377955
3000:2.9211226746920946
4000:1.4545267751242743
5000:0.5760406074970698
6000:0.1564464089438815
7000:0.02188090874378457
8000:0.000930437905488198
9000:4.95661682923231e-06
10000:1.0486557353663985e-09
11000:2.549340742887293e-10
12000:1.9911096254524357e-10
13000:1.7776737168323377e-10
14000:1.7051878583847647e-10
15000:1.678251511874035e-10
16000:1.6687260302741238e-10
17000:1.6652622167679437e-10
18000:1.6643962622754643e-10
19000:1.6640714347775107e-10
20000:1.6683150664009982e-10


In [6]:
###提取模型求解出来的参数并打印####
alpha = (model.alpha).detach().item() #打印模型参数，与v相关的阻力系数
beta = (model.beta).detach().item() #打印模型参数,与s相关的阻力系数
print("alpha:{}".format(alpha))
print("beta:{}".format(beta))

alpha:2.9701883792877197
beta:0.9884071350097656


In [7]:
####使用odient库求解微分方程的方法
def v_func(sv, t, alpha, beta):  #定义一个微分方程，输入分别为sv（一个数对，分别为位移s和速度v；t为时间，直接这么写就行，alpha，beta分别是与v有关的阻力系数和与s有关的阻力系数）
    s,v = sv 
    a = - alpha * v - beta * s #微分方程
    return np.array([v,a])
sv_expect = odeint(v_func, [s[-1],v[-1]], [0,5], args=(alpha,beta))  #打印五秒后的位移和速度，其中s[-1]和v[-1]提取最后一个数，表示提取当前位置的数，0表示当前时刻，5表示5秒后
print("再过5秒后的位移s为",sv_expect[1,0]) #第一列为位移，第一行为当前时刻的位移
print("再过5秒后的速度v为",sv_expect[1,1]) #第二列为速度，第二行为当前时刻的速度

再过5秒后的位移s为 0.1077077506160201
再过5秒后的速度v为 -0.04113065392973712
